# Model a Galaxy Cluster

## Notebook for generating an example galaxy cluster model. 

This notebook goes through the steps to generate model data for galaxy cluster weak lensing observables.  In particular, we define a galaxy cluster model that follows and NFW distribution and generate various profiles for the model (mass density, convergence, shear, etc.), which we plot.  Note, a full pipeline to measure a galaxy cluster weak lensing mass requires fitting the observed (or mock) data to a model. In this notebook we use the OO interface to theory.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Imports specific to clmm 

In [ ]:
import os

os.environ['CLMM_MODELING_BACKEND'] = 'nc' # here you may choose ccl, nc (NumCosmo) or ct (cluster_toolkit)

import clmm
from clmm import Cosmology

Make sure we know which version we're using

In [ ]:
clmm.__version__

Define a cosmology using astropy

In [ ]:
cosmo = Cosmology(H0=70.0, Omega_dm0=0.27-0.045, Omega_b0=0.045, Omega_k0=0.0)

Define the galaxy cluster model.  Here, we choose parameters that describe the galaxy cluster model, including the mass definition, concentration, and mass distribution.  For the mass distribution, we choose a distribution that follows an NFW profile.

In [ ]:
moo = clmm.Modeling(massdef='mean', delta_mdef=200, halo_profile_model='nfw')

moo.set_cosmo(cosmo)
moo.set_concentration(4)
moo.set_mass(1.e15)

z_cl = 1.
z_source = 2.
alpha = [2, -0.5] 
halo_bias = 1.


Quick test of all theory functionality

In [ ]:
r3d = np.logspace(-2, 2, 100)
rho = moo.eval_3d_density(r3d, z_cl)
Sigma = moo.eval_surface_density(r3d, z_cl)
DeltaSigma = moo.eval_excess_surface_density(r3d, z_cl)
Sigmac = moo.eval_critical_surface_density(z_cl, z_source)
gammat = moo.eval_tangential_shear(r3d, z_cl, z_source)
kappa = moo.eval_convergence(r3d, z_cl, z_source)
gt = moo.eval_reduced_tangential_shear(r3d, z_cl, z_source)
mu = moo.eval_magnification(r3d, z_cl, z_source)
mu_bias = moo.eval_magnification_bias(r3d, z_cl, z_source, alpha)

Plot the predicted profiles

In [ ]:
def plot_profile(r, profile_vals, profile_label='rho', label = ''):
    plt.loglog(r, profile_vals, label=label)
    plt.xlabel('r [Mpc]', fontsize='xx-large')
    plt.ylabel(profile_label, fontsize='xx-large')

In [ ]:
plot_profile(r3d, rho, '$\\rho_{\\rm 3d}$')

In [ ]:
plot_profile(r3d, Sigma, '$\\Sigma_{\\rm 2d}$')

In [ ]:
plot_profile(r3d, DeltaSigma, '$\\Delta\\Sigma_{\\rm 2d}$')

In [ ]:
plot_profile(r3d, kappa, '$\\kappa$')

In [ ]:
plot_profile(r3d, gammat, '$\\gamma_t$')

In [ ]:
plot_profile(r3d, gt, '$g_t$')

In [ ]:
plot_profile(r3d, mu, '$\mu$')

In [ ]:
plot_profile(r3d, mu_bias[0]-1, profile_label = '$\delta_{\mu}$', label = '$\\alpha$ =' + str(alpha[0]))
plot_profile(r3d, mu_bias[1]-1, '$\delta_{\mu}$', label = '$\\alpha$ =' + str(alpha[1]))

plt.legend(fontsize='xx-large')
plt.yscale('linear')
plt.grid()

plt.ylim(-3,5)

In [ ]:
# The 2-halo term excess surface density is currently only implemented for the CCL backend
# An error will be raised if using the CT or NC backend instead

DeltaSigma_2h = moo.eval_excess_surface_density_2h(r3d, z_cl, halo_bias=halo_bias)
plot_profile(r3d, DeltaSigma_2h, '$\\Delta\\Sigma_{\\rm 2h}$')